In [1]:
!nvidia-smi
# 아무것도 안넣은 결과
# smart pad 결과
# 시형님 데이터 샘플링 코드 적용 결과


Sat Oct 22 13:41:14 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 520.61.03    Driver Version: 522.06       CUDA Version: 11.8     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0 Off |                  N/A |
| N/A   50C    P8     5W /  N/A |      0MiB /  6144MiB |      2%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import glob
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import argparse
import random
import os, sys
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from tqdm import tqdm

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

import torchvision.models as models

from sklearn import preprocessing
sys.path.append('../') # import를 위해 경로추가
from Modules import Utility as U
from Modules import CustomDataset 
from Modules import SmartPad
#torch.backends.cudnn.enabled = False

In [3]:
parser = argparse.ArgumentParser()
parser.add_argument('--epochs', type=int, default=20)
parser.add_argument('--lr', type=float, default=0.001)
parser.add_argument('--batch_size', type=int, default=10)
parser.add_argument('--seed', type=int, default=41)
parser.add_argument('--img_size', type=int, default=300)
parser.add_argument('--device', default='cuda')
parser.add_argument('--data_path', type=str, default='../data/')
parser.add_argument('--save_name', type=str, default="effnet_b4_300_smart_pad")

args = parser.parse_args('')
args

Namespace(batch_size=10, data_path='../data/', device='cuda', epochs=20, img_size=300, lr=0.001, save_name='effnet_b4_300_smart_pad', seed=41)

In [4]:
U.seed_everything(args.seed)

In [5]:
df = pd.read_csv(args.data_path+'train_repaired.csv')

In [6]:
le = preprocessing.LabelEncoder()
df['artist'] = le.fit_transform(df['artist'].values)
df.head()

,id,img_path,artist
0,0,./train/0000.jpg,9
1,1,./train/0001.jpg,48
2,2,./train/0002.jpg,7
3,3,./train/0003.jpg,10
4,4,./train/0004.jpg,24


In [7]:
train_df, val_df, _, _ = U.train_test_split(df, df['artist'].values, test_size=0.2, random_state=args.seed)

In [8]:
x = np.zeros((3,50))
for a in df['artist']:
    x[0,a] += 1
for a in train_df['artist']:
    x[1,a] += 1
for a in val_df['artist']:
    x[2,a] += 1
print(x)

[[220. 165. 132.  74. 132.  64.  32.  59.  50.  81. 489.  62.  44.  65.
   26. 204.  84.  30.  72.  69.  42. 121.  52.  61. 115.  21.  64.  76.
   91. 101. 173.  34. 118. 303.  33. 220. 142.  97. 233.  59.  85.  73.
  181. 137.  99. 120. 173.  60. 629.  44.]
 [175. 134. 110.  63. 103.  53.  28.  50.  38.  68. 389.  51.  30.  58.
   17. 167.  68.  21.  61.  58.  32.  93.  41.  45.  95.  16.  46.  59.
   75.  79. 137.  28. 101. 250.  30. 178. 107.  81. 185.  47.  69.  58.
  141. 111.  75.  88. 129.  49. 502.  39.]
 [ 45.  31.  22.  11.  29.  11.   4.   9.  12.  13. 100.  11.  14.   7.
    9.  37.  16.   9.  11.  11.  10.  28.  11.  16.  20.   5.  18.  17.
   16.  22.  36.   6.  17.  53.   3.  42.  35.  16.  48.  12.  16.  15.
   40.  26.  24.  32.  44.  11. 127.   5.]]


In [9]:
train_img_paths, train_labels = U.get_data(train_df)
val_img_paths, val_labels = U.get_data(val_df)

In [10]:
# random flip
train_transform = A.Compose([
                            SmartPad(),
                            A.Resize(args.img_size, args.img_size),
                            # A.RandomResizedCrop(height = args.img_size, 
                            #                     width = args.img_size,
                            #                     scale = (0.8,1)),
                            # A.HorizontalFlip(),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

test_transform = A.Compose([
                            SmartPad(),
                            A.Resize(args.img_size,args.img_size),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

In [11]:
train_dataset = CustomDataset(args.data_path, train_img_paths, train_labels, train_transform)
train_loader = DataLoader(train_dataset, batch_size = args.batch_size, shuffle=True, num_workers=0)

val_dataset = CustomDataset(args.data_path, val_img_paths, val_labels, test_transform)
val_loader = DataLoader(val_dataset, batch_size=args.batch_size, shuffle=False, num_workers=0)

In [12]:
class CustomModel(nn.Module):
    def __init__(self, number_of_classes :int):
        super(CustomModel, self).__init__()
        self.backborn = models.efficientnet_b4(weights=models.EfficientNet_B4_Weights.DEFAULT)
        # for p in self.backborn.parameters():
        #     p.requires_grad = False
        self.backborn.classifier = nn.Sequential(
            nn.Linear(1792, 512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(0.2),
            nn.Dropout(p=0.2),
            nn.Linear(512, number_of_classes)
        )
        
    def forward(self, x):
        x = self.backborn(x)
        return x

In [13]:
target_model = CustomModel(50)

In [14]:
#for p in target_model.parameters():
#    p.requires_grad = False
#for p in target_model.classifier[0].parameters():
#    p.requires_grad = True
#target_model.classifier[1] = torch.nn.Linear(in_features=1280, out_features=50, bias=True)

In [15]:
def validation(model, criterion, test_loader, device):
    model.eval()
    
    model_preds = []
    true_labels = []
    
    val_loss = []
    
    with torch.no_grad():
        for item in tqdm(test_loader):
            img = item['image']
            label = item['label']
            h = item['height']
            w = item['width']
            img, label = img.float().to(device), label.to(device)
            model_pred = model(img)
            
            loss = criterion(model_pred, label)
            
            val_loss.append(loss.item())
            
            model_preds += model_pred.argmax(1).detach().cpu().numpy().tolist()
            true_labels += label.detach().cpu().numpy().tolist()
        
    val_f1 = U.competition_metric(true_labels, model_preds)
    return np.mean(val_loss), val_f1

In [16]:
def train(model, optimizer, train_loader, test_loader, scheduler, device):
    model.to(device)

    criterion = nn.CrossEntropyLoss().to(device)
    
    best_score = 0
    best_model = None
    
    for epoch in range(1, args.epochs+1):
        model.train()
        train_loss = []
        for item in tqdm(train_loader):
            img = item['image']
            label = item['label']
            h = item['height']
            w = item['width']
            img, label = img.float().to(device), label.to(device)
            optimizer.zero_grad()

            model_pred = model(img)
            
            loss = criterion(model_pred, label)

            loss.backward()
            optimizer.step()

            train_loss.append(loss.item())

        tr_loss = np.mean(train_loss)
            
        val_loss, val_score = validation(model, criterion, test_loader, device)
            
        print(f'Epoch [{epoch}], Train Loss : [{tr_loss:.5f}] Val Loss : [{val_loss:.5f}] Val F1 Score : [{val_score:.5f}]')
        
        if scheduler is not None:
            scheduler.step()
            
        if best_score < val_score:
            best_model = model
            best_score = val_score
        
    return best_model

In [17]:
model = target_model.to(args.device)
model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = args.lr)

scheduler = None
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

infer_model = train(model, optimizer, train_loader, val_loader, scheduler, args.device)

100%|██████████| 119/119 [00:47<00:00,  2.49it/s]


Epoch [1], Train Loss : [2.08211] Val Loss : [1.35576] Val F1 Score : [0.50346]


100%|██████████| 119/119 [00:47<00:00,  2.51it/s]


Epoch [2], Train Loss : [1.03817] Val Loss : [1.15813] Val F1 Score : [0.59334]


100%|██████████| 119/119 [00:49<00:00,  2.39it/s]


...

Epoch [18], Train Loss : [0.15997] Val Loss : [2.02740] Val F1 Score : [0.62910]


100%|██████████| 119/119 [00:46<00:00,  2.53it/s]


Epoch [19], Train Loss : [0.17308] Val Loss : [1.60704] Val F1 Score : [0.62555]


100%|██████████| 119/119 [00:48<00:00,  2.47it/s]

Epoch [20], Train Loss : [0.11930] Val Loss : [1.42353] Val F1 Score : [0.65221]


In [18]:
torch.save({
    'model': infer_model.state_dict(),
    'optim': optimizer.state_dict()
}, args.save_name + '.tar')

In [19]:
test_df = pd.read_csv(args.data_path + 'test.csv')

In [20]:
test_img_paths = U.get_data(test_df, infer=True)

In [21]:
test_dataset = CustomDataset(args.data_path, test_img_paths, None, test_transform)
test_loader = DataLoader(test_dataset, batch_size=args.batch_size, shuffle=False, num_workers=0)

In [22]:
def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    
    model_preds = []
    
    with torch.no_grad():
        for item in tqdm(iter(test_loader)):
            img = item['image'].float().to(device)
            h = item['height']
            w = item['width']
            
            model_pred = model(img)
            model_preds += model_pred.argmax(1).detach().cpu().numpy().tolist()
    
    print('Done.')
    return model_preds

In [23]:
preds = inference(infer_model, test_loader, args.device)

 10%|█         | 132/1267 [00:36<08:24,  2.25it/s]

In [ ]:
preds = le.inverse_transform(preds) 

In [ ]:
submit = pd.read_csv(args.data_path + 'sample_submission.csv')

In [ ]:
submit['artist'] = preds
submit.head()

In [ ]:
submit.to_csv('./' + args.save_name + '.csv', index=False)

In [ ]:
torch.cuda.memory_reserved()

In [ ]:
torch.cuda.empty_cache()